## **Отчет по проекту "Речевая аналитика на основе ИИ"**

---



# Команда: Жекенбай Али , Пернебек Бакберген



# **Введение**

Цель проекта — разработать систему речевой аналитики на основе технологий искусственного интеллекта. Система использует аудиофайлы для извлечения текста, анализа эмоций, тональности речи, суммаризации текста и выделения ключевых слов/фраз. В качестве основных технологий применяются Whisper для транскрибации аудио в текст и платформа Hugging Face, используя модели такие как KeyBERT, SentenceTransformer, и архитектуру Transformers для анализа текста.

# **Проблема**

*   Низкий уровень контроля качества звонков руководителями.
*   Долгий процесс прослушивания аудиозвонков.
*   Потеря критически важной информации из разговоров

# **Решение**

*  Речевая аналитика на базе ИИ — позволяет компаниям с клиентским сервисом делать 100% анализ аудио-видеозвонков и извлекать из них ценную информацию на основе ИИ.

# **Актуальность**

*  ИИ-речевая аналитика выявляет эмоции, тональность и проблемы в звонках, недоступные при ручном анализе, помогая компаниям улучшать качества обслуживания и повышать операционую эффективность.

# **Значимость**

*  Инструмент на базе ИИ предоставляет возможность автоматической обработки больших объемов данных с минимальными затратами времени и ресурсов. Это повышает эффективность взаимодействия с клиентами, улучшает обучение сотрудников и способствует росту выручки.

# **Этапы реализаций проекта:**

Загружаем библиотеку Whisper для обработки аудио и транскрибации.

In [1]:
pip install git+https://github.com/openai/whisper.git openai

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-a2719ohc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-a2719ohc
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803583 sha256=d3836ec7d1aa8a0bfe0e7c0f901ff13953ff7ceb7817e22c74f2761df0e70ac2
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_p6jsw8/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


**Устанавливаем библиотеки для работы с аудио, анализа текста, извлечения ключевых слов и отображения прогресса:**


1.   *openai:* работа с API OpenAI.
2.   *torch:* нейросети и обработка данных.
2.   *tqdm:* прогресс операций.
2.   *transformers:* анализ текста.
2.   *sentence-transformers:* поиск ключевых фраз.
2.   *keybert:* выделение ключевых слов.











In [2]:
pip install openai torch tqdm transformers sentence-transformers keybert

Подключаем Google Drive для хранения аудиофайлов и результатов анализа:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Код выполняет полный цикл обработки аудио для анализа речи и текста, включая транскрибацию, анализ текста, извлечение ключевых слов и сохранение результатов.

In [4]:
import torch
import whisper
from tqdm import tqdm
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from keybert import KeyBERT
import sys

# Параметры
audio = "We Got Married!.mp3"  # Загружаем аудиофайл!
model_type = "base"  # Доступные модели: 'tiny', 'base', 'small', 'medium', 'large'

# Функция транскрибации
def transcribe(audio, model_type):
    devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = whisper.load_model(model_type, device=devices)

    print("Начало транскрибации...")
    result = model.transcribe(audio, verbose=False, fp16=torch.cuda.is_available())
    transcribed = result["text"]

    with open("Transcript.txt", "w", encoding='utf-8') as text_file:
        text_file.write(transcribed)
        print("Транскрипция сохранена в Transcript.txt")
    return transcribed

# Извлечение ключевых слов с помощью KeyBERT
def extract_keywords_keybert(text, top_n=5):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text, top_n=top_n)
    return keywords

# Извлечение ключевых фраз через Semantic Search
def extract_keywords_semantic(text, top_k=5):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    sentences = text.split(". ")  # Разделение на предложения
    embeddings = model.encode(sentences)
    query_embedding = model.encode(text)
    search_results = util.semantic_search(query_embedding, embeddings, top_k=top_k)

    keywords = [sentences[hit['corpus_id']] for hit in search_results[0]]
    return keywords

# Анализ текста
def analyze_text(text):
    print("Начало анализа текста...")

    # Анализ эмоций
    emotion_classifier = pipeline("sentiment-analysis")
    emotions = emotion_classifier(text)

    # Анализ тональности речи
    tone_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    candidate_labels = ["formal", "informal", "neutral", "emotional"]
    tone_analysis = tone_classifier(text, candidate_labels=candidate_labels)

    # Суммаризация текста
    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)

    # Извлечение ключевых слов с использованием KeyBERT
    keywords_keybert = extract_keywords_keybert(text)

    # Извлечение ключевых фраз через Semantic Search
    keywords_semantic = extract_keywords_semantic(text)

    # Сохранение результата анализа
    with open("Analysis.txt", "w", encoding='utf-8') as file:
        file.write("Оригинальный текст:\n" + text + "\n\n")
        file.write("Анализ эмоций:\n" + str(emotions) + "\n\n")
        file.write("Анализ тональности:\n" + str(tone_analysis) + "\n\n")
        file.write("Суммаризация текста:\n" + summary[0]['summary_text'] + "\n\n")
        file.write("Ключевые слова (KeyBERT):\n" + str(keywords_keybert) + "\n\n")
        file.write("Ключевые фразы (Semantic Search):\n" + str(keywords_semantic) + "\n")
        print("Анализ сохранён в Analysis.txt")

# Главная функция: транскрибация и анализ
if __name__ == "__main__":
    transcribed_text = transcribe(audio, model_type)
    analyze_text(transcribed_text)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 156MiB/s]


Начало транскрибации...
Detected language: English


100%|██████████| 6617/6617 [00:16<00:00, 411.64frames/s]
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Транскрипция сохранена в Transcript.txt
Начало анализа текста...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Анализ сохранён в Analysis.txt


# **Заключение**

Проект "Речевая аналитика на основе ИИ" продемонстрировал, как современные технологии ИИ могут эффективно преобразовывать аудио в текст и выполнять глубокий анализ полученной информации. Мы успешно реализовали полный цикл обработки данных, включая транскрибацию речи, извлечение ключевых слов, определение эмоционального окраса, анализ тональности и создание кратких резюме.

Этот инструмент найдет широкое применение в бизнесе, образовании, здравоохранении и других сферах, где требуется автоматизация анализа речевых данных. Наш проект стал примером того, как искусственный интеллект может повышать точность и скорость анализа, минимизируя человеческие затраты.